In [1]:
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as f
import csv 
import random
import re
import os
import unicodedata
import codecs
import itertools

In [2]:
CUDA = torch.cuda.is_available()
device = torch.device("cuda" if CUDA else "cpu")

# Data Preprocessing

In [3]:
lines_filepath = os.path.join("cornell movie-dialogs corpus","movie_lines.txt")
conv_filepath = os.path.join("cornell movie-dialogs corpus","movie_conversations.txt")

In [4]:
# Visualize some lines
with open(lines_filepath,'r',encoding="iso-8859-1") as file:
    lines = file.readlines()
    for line in lines[:8]:
        #print(line.strip())
        pass

In [5]:
# Splitting each line of the file into a dictioanry of fields (LineId,CharacterId,MovieId,Character,Text)
line_fields = ["lineId","characterId","movieId","character","text"]
lines = {}
with open(lines_filepath,'r',encoding="iso-8859-1") as f:
    for line in f:
        values = line.split(" +++$+++ ")
        # Extract fileds
        lineobj = {}
        for i,field in enumerate(line_fields):
            lineobj[field] = values[i]
            #print(lineobj)
        lines[lineobj['lineId']] = lineobj
        #print(lines)

In [6]:
# Groups fields of lines from 'Loadlines' into conversations based on "movie_conversations.txt"
conv_fields = ["character1Id","character2Id","movieId","utterenceIDs"]
conversations = []
with open(conv_filepath,'r',encoding='iso-8859-1') as f:
    for line in f:
        values = line.split(" +++$+++ ")
        # Extract Fields
        convObj = {}
        for i,field in enumerate(conv_fields):
            convObj[field] = values[i]
        # Convert string result from split to list, since convObj["utterenceIDs"] == "["L598765","L567890","...."]"
        lineIds = eval(convObj["utterenceIDs"])
        #print(lineIds)
        # Reassemble lines
        convObj["lines"] = []
        for lineId in lineIds:
            convObj["lines"].append(lines[lineId])
        conversations.append(convObj)

In [7]:
# Extract pairs of sentences from conversations
qa_pairs = []
for conversation in conversations:
    # Iterate over all the lines of conversation
    for i in range(len(conversation["lines"]) - 1):
        inputLine = conversation["lines"][i]["text"].strip()
        targetLine = conversation["lines"][i+1]["text"].strip()
        # Filter wrong samples if one of the lists is empty
        if inputLine and targetLine:
            qa_pairs.append([inputLine,targetLine])

In [8]:
# Define path to new file
datafile = os.path.join("cornell movie-dialogs corpus","formatted_movie_lines.txt")
delimiter = '\t'
# Unescape the delimiter
delimiter = str(codecs.decode(delimiter,"unicode_escape"))

# write new csv file
print("\n writing newly formatted file")
with open(datafile,"w",encoding='utf-8') as outputfile:
    writer = csv.writer(outputfile,delimiter=delimiter)
    for pair in qa_pairs:
        #print(pair[0])
        writer.writerow(pair)
print("Done writing to the file ")


 writing newly formatted file
Done writing to the file 


In [9]:
# Visualize some Lines
datafile = os.path.join("cornell movie-dialogs corpus","formatted_movie_lines.txt")
with open (datafile,'r',encoding="iso-8859-1") as file:
    lines = file.readlines()
    for line in lines[:8]:
        print(line)

Can we make this quick?  Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad.  Again.	Well, I thought we'd start with pronunciation, if that's okay with you.

Well, I thought we'd start with pronunciation, if that's okay with you.	Not the hacking and gagging and spitting part.  Please.

Not the hacking and gagging and spitting part.  Please.	Okay... then how 'bout we try out some French cuisine.  Saturday?  Night?

You're asking me out.  That's so cute. What's your name again?	Forget it.

No, no, it's my fault -- we didn't have a proper introduction ---	Cameron.

Cameron.	The thing is, Cameron -- I'm at the mercy of a particularly hideous breed of loser.  My sister.  I can't date until she does.

The thing is, Cameron -- I'm at the mercy of a particularly hideous breed of loser.  My sister.  I can't date until she does.	Seems like she could get a date easy enough...

Why?	Unsolved mystery.  She used to be really popular when she started h

In [10]:
PAD_TOKEN = 0
SOS_TOKEN = 0
EOS_TOKEN = 2

class Vocabulary:
    def __init__(self,name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_TOKEN:"PAD",SOS_TOKEN:"SOS",EOS_TOKEN:"EOS"}
        self.num_words = 3
        
    def addSentence(self,sentence):
        for word in sentence.split(' '):
            self.addWord(word)
            
    def addWord(self,word):
        if word not in self.word2index:
            self.word2index[word] = self.num_words
            self.word2count[word] = 1
            self.index2word[self.num_words] = word
            self.num_words += 1
        else:
            self.word2count[word] += 1
            
    # Remove words below a certain count threshold
    def trim(self, min_count):
        keep_words = []
        for k,v in self.word2count.items():
            if v >= min_count:
                keep_words.append(k)
                
        print('keep_words {}/{} = {:.4f}'.format(len(keep_words),len(self.word2index),len(keep_words)/len(self.word2index)))
    
        # Reinitialize the dictionaries 
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_TOKEN:"PAD",SOS_TOKEN:"SOS",EOS_TOKEN:"EOS"}
        self.num_words = 3 
        
        for word in keep_words:
              self.addWord(word)
              

In [11]:
# Turn a unicode string to plain ASCII
import unicodedata
def unicodeToAscii(s):
    return ''.join(c for c in unicodedata.normalize('NFD',s) if unicodedata.category(c) != 'Mn')

In [12]:
''.join(['h','e','l'])

'hel'

In [13]:
unicodeToAscii("Adiós,Pequeño....")

'Adios,Pequeno....'

In [14]:
# Lowercase, trim white spaces, lines... etc, and remove non letters character.
import re
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    # Replace any .!? by a whitespace  + the character --> '!' = ' !'. \1 means that first bracketed group -->[,!] .
    # r is to not consider \1 as a character (r to escape a backslash). + means one or more
    s = re.sub(r"([.!?])",r" \1",s)
    # Remove character that is not a sequence of lower or uppercase
    s = re.sub(r"[^a-zA-Z.!?]+",r" ",s)
    # Remove a sequence of whitespace character
    s = re.sub(r"\s+",r" ",s).strip()
    return s

In [15]:
normalizeString("aaa123!s's   dd?")

'aaa !s s dd ?'

In [16]:
import os
datafile = os.path.join("cornell movie-dialogs corpus","formatted_movie_lines.txt")
# Read the files and split into lines
print("Reading and Processing the file........... please wait")
lines = open(datafile,encoding="iso-8859-1").read().strip().split('\n')
# Split every line into pairs and normalize
pairs = [[normalizeString(s) for s in pair.split('\t')] for pair in lines]
print("Done Reading !!!")
voc = Vocabulary("cornell movie-dialogs corpus")

Reading and Processing the file........... please wait
Done Reading !!!


In [17]:
lines[0].split('\t')

['Can we make this quick?  Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad.  Again.',
 "Well, I thought we'd start with pronunciation, if that's okay with you."]

In [18]:
# Returns true if both sentences in a pair 'p' are under the max length threshold
MAX_LENGTH = 10 # Maximum sentence length to consider
def filterPair(p):
    # Input sequences need to preserve the last word for EOS token
    return len(p[0].split()) < MAX_LENGTH and len(p[1].split()) < MAX_LENGTH

# Filter pairs using filter pair condition
def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [19]:
pairs = [pair for pair in pairs if len(pair) > 1]
print("There are {} pairs/conversations in the dataset ".format(len(pairs)))
pairs = filterPairs(pairs)
print("After filtering there are {} pairs/conversations ".format(len(pairs)))

There are 221282 pairs/conversations in the dataset 
After filtering there are 64243 pairs/conversations 


In [20]:
# Loop through each pair and add the question and reply e=sentence to the vocabulary
for pair in pairs:
    voc.addSentence(pair[0])
    voc.addSentence(pair[1])
print("counted words :", voc.num_words)
for pair in pairs[:10]:
    print(pair)
    

counted words : 18109
['there .', 'where ?']
['you have my word . as a gentleman', 'you re sweet .']
['hi .', 'looks like things worked out tonight huh ?']
['you know chastity ?', 'i believe we share an art instructor']
['have fun tonight ?', 'tons']
['well no . . .', 'then that s all you had to say .']
['then that s all you had to say .', 'but']
['but', 'you always been this selfish ?']
['do you listen to this crap ?', 'what crap ?']
['what good stuff ?', 'the real you .']


In [21]:
MIN_COUNT = 3   # minimum word count threshold for trimming
def trimRareWords(voc,pairs,MIN_COUNT):
    # Trim words used under the MIN_COUNT from the voc
    voc.trim(MIN_COUNT)
    # filter out paired with trimmed words
    keep_pairs = []
    for pair in pairs:
        input_sentence = pair[0]
        output_sentence = pair[1]
        keep_input = True
        keep_ourput = True
        
        # Check your sentence 
        for word in input_sentence.split(' '):
            if word not in voc.word2index:
                keep_input = False
                break
                
        # Check output sentence
        for word in output_sentence.split(' '):
            if word not in voc.word2index:
                keep_output = False
                break
                
        # only keep pairs that do not contain trimmed word(s) in their input or output sentence
        if keep_input and keep_ourput:
            keep_pairs.append(pair)
            
    print("Trimmed from {} pairs to {},{:.4f} of total ".format(len(pairs),len(keep_pairs),len(keep_pairs)/len(pairs)))
    return keep_pairs

# Print voc and pairs
pairs = trimRareWords(voc,pairs,MIN_COUNT)

keep_words 7840/18106 = 0.4330
Trimmed from 64243 pairs to 57971,0.9024 of total 


# Preparing the data

In [22]:
def indexFromSentence(voc,sentence):
    return [voc.word2index[word] for word in sentence.split(' ')] + [EOS_TOKEN]

In [23]:
pairs[1][0]

'you have my word . as a gentleman'

In [24]:
# Test the function
indexFromSentence(voc,pairs[1][0])

[7, 8, 9, 10, 4, 11, 12, 13, 2]

In [25]:
# Define some samples for output
inp = []
out = []
for pair in pairs[:10]:
    inp.append(pair[0])
    out.append(pair[1])
print(inp)
print(len(inp))
indexes = [indexFromSentence(voc,sentence) for sentence in inp]
indexes

['there .', 'you have my word . as a gentleman', 'hi .', 'have fun tonight ?', 'well no . . .', 'then that s all you had to say .', 'but', 'do you listen to this crap ?', 'what good stuff ?', 'the real you .']
10


[[3, 4, 2],
 [7, 8, 9, 10, 4, 11, 12, 13, 2],
 [16, 4, 2],
 [8, 31, 22, 6, 2],
 [33, 34, 4, 4, 4, 2],
 [35, 36, 37, 38, 7, 39, 40, 41, 4, 2],
 [42, 2],
 [47, 7, 48, 40, 45, 49, 6, 2],
 [50, 51, 52, 6, 2],
 [53, 54, 7, 4, 2]]

In [26]:
def zeroPadding(l,fillvalue=0):
    return list(itertools.zip_longest(*l,fillvalue=fillvalue))

In [27]:
leng = [len(ind) for ind in indexes]
max(leng)

10

In [28]:
# Test the function
test_result = zeroPadding(indexes)
print(len(test_result)) # The max length is now the number of rows or the batch size
test_result

10


[(3, 7, 16, 8, 33, 35, 42, 47, 50, 53),
 (4, 8, 4, 31, 34, 36, 2, 7, 51, 54),
 (2, 9, 2, 22, 4, 37, 0, 48, 52, 7),
 (0, 10, 0, 6, 4, 38, 0, 40, 6, 4),
 (0, 4, 0, 2, 4, 7, 0, 45, 2, 2),
 (0, 11, 0, 0, 2, 39, 0, 49, 0, 0),
 (0, 12, 0, 0, 0, 40, 0, 6, 0, 0),
 (0, 13, 0, 0, 0, 41, 0, 2, 0, 0),
 (0, 2, 0, 0, 0, 4, 0, 0, 0, 0),
 (0, 0, 0, 0, 0, 2, 0, 0, 0, 0)]

In [29]:
def binaryMatrix(l,value=0):
    m = []
    for i,seq in enumerate(l):
        m.append([])
        for token in seq:
            if token in seq:
                if token == PAD_TOKEN:
                    m[i].append(0)
                else:
                    m[i].append(1)
    return m

In [30]:
binary_result = binaryMatrix(test_result)
binary_result

[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 0, 1, 1, 1],
 [0, 1, 0, 1, 1, 1, 0, 1, 1, 1],
 [0, 1, 0, 1, 1, 1, 0, 1, 1, 1],
 [0, 1, 0, 0, 1, 1, 0, 1, 0, 0],
 [0, 1, 0, 0, 0, 1, 0, 1, 0, 0],
 [0, 1, 0, 0, 0, 1, 0, 1, 0, 0],
 [0, 1, 0, 0, 0, 1, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 1, 0, 0, 0, 0]]

In [31]:
# Returns padded input sequence tensor and as well as a tensor of length for each of the sequence in batch
def inputVar(l,voc):
    indexes_batch = [indexFromSentence(voc,sentence) for sentence in l]
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    padList = zeroPadding(indexes_batch)
    padVar = torch.LongTensor(padList)
    return padVar,lengths

In [32]:
# Returns padded target sequence tensor, padding mask and max target length
def outputVar(l,voc):
    indexes_batch = [indexFromSentence(voc,sentence) for sentence in l]
    max_target_len = max([len(indexes) for indexes in indexes_batch])
    padList = zeroPadding(indexes_batch)
    mask = binaryMatrix(padList)
    mask = [torch.ByteTensor(mask)]
    padVar = torch.LongTensor(padList)
    return padVar,mask,max_target_len

In [33]:
# Returns all items for a given batch of pairs
def batch2TrainData(voc,pair_batch):
    # sort the question in descending length
    pair_batch.sort(key=lambda x: len(x[0].split(" ")),reverse=True)
    input_batch,output_batch = [],[]
    for pair in pair_batch:
        input_batch.append(pair[0])
        output_batch.append(pair[1])
    inp,lengths = inputVar(input_batch,voc) 
    output,mask,max_target_len = outputVar(output_batch,voc)
    return inp,lengths,output,mask,max_target_len

In [34]:
# Example for validation
small_batch_size = 5
batches = batch2TrainData(voc,[random.choice(pairs) for _ in range(small_batch_size)])
input_variable,lengths,target_variable,mask,max_target_len = batches

print("Input_variable :")
print(input_variable)
print("lengths",lengths)
print("target_variable :")
print(target_variable)
print("mask :")
print(mask)
print("max_target_len",max_target_len)


Input_variable :
tensor([[  50,    7,  997,   34,  742],
        [  47,   39,    4,   50,    6],
        [   7, 2554,    4,    6,    2],
        [  24,  720,    4,    2,    0],
        [  56, 1110,    6,    0,    0],
        [   9,    6,    2,    0,    0],
        [1436,    2,    0,    0,    0],
        [   6,    0,    0,    0,    0],
        [   2,    0,    0,    0,    0]])
lengths tensor([9, 7, 6, 4, 3])
target_variable :
tensor([[  59,  124, 3232,   53,   53],
        [ 386, 3023,    4, 2487,  301],
        [ 397,    4,    4,    4,  140],
        [  76, 6276,    4,    2,    4],
        [ 843,  782,    6,    0,    2],
        [   4,  492,    2,    0,    0],
        [   2, 2330,    0,    0,    0],
        [   0,    4,    0,    0,    0],
        [   0,    2,    0,    0,    0]])
mask :
[tensor([[1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 0, 1],
        [1, 1, 1, 0, 0],
        [1, 1, 0, 0, 0],
        [0, 1, 0, 0, 0],
   

In [35]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class EncoderRNN(nn.Module):
    def __init__(self,hidden_size,embedding,n_layers=1,dropout=0):
        super(EncoderRNN,self).__init__()
        self.n_layers = n_layers
        self.hidden_size = hidden_size
        self.embedding = embedding
        # Initialize GRU; the input_size and hidden_size params are bith set to 'hidden_size'
        # because our input size is a word embedding with numner of features == hidden_size
        self.gru = nn.GRU(hidden_size,hidden_size,n_layers,dropout=(0 if n_layers == 1 else dropout),bidirectional=True)

In [36]:
def forward(self,input_seq,input_lengths,hidden=None):
    # input_seq : batch of input sentences; shape=(max_length,batch_size)
    # input_lengths : List of sentence Lengths corresponding to each sentence in a batch
    # hidden state, of shape: (n_layers x num_directions,batch_size,hidden_size)
    # convert word indexes to embeddings
    embedded = self.embedding(input_seq)
    # pack padded batch of sequences for RNN module
    packed = torch.nn.utils.rnn.pack_added_sequence(embedded,input_lengths)
    # Forward pass through GRU
    outputs,hidden = self.gru(packed,hidden)
    # unpack padding
    outputs,_ = torch.nn.utils.rnn.pad_packed_sequence(outputs)
    # Sum bidirectional GRU outputs
    outputs = outputs[:,:,:self.hidden_size] + outputs[:,:,self.hidden_size]
    # Return output and final hidden state
    # outputs: the output features h_t from the last Layer of the GRU, for each timestep (sum of bidirectional outputs)
    # outputs shape=(max_length,batch_size,hidden_size)
    # hidden: hidden state for the last timestep , of shape=(n_layers x num_directions,batch_size,hidden_size)
    

In [37]:
import torch
a = torch.randn(5,4,3)
a

tensor([[[ 0.6884,  0.6820, -1.2640],
         [ 0.2292,  1.0449, -1.1365],
         [-0.5910, -0.7369, -0.6018],
         [ 0.4569,  0.7105, -0.0465]],

        [[-0.8110,  1.1275,  2.0840],
         [-0.6609, -0.2096, -0.0190],
         [ 0.0952, -0.0245,  0.4777],
         [-0.0971, -0.7193, -0.5996]],

        [[ 0.3454, -0.6712, -0.9245],
         [-0.3776, -0.0202, -0.3335],
         [-0.1226, -2.2257, -0.6050],
         [-1.8096, -0.6503, -1.1710]],

        [[ 0.5572, -0.0194, -0.1122],
         [-0.3975, -1.4546, -0.6804],
         [-0.7679,  1.1416, -0.7831],
         [ 2.2972,  1.7127, -0.0985]],

        [[-1.7782, -1.8316,  0.9600],
         [ 0.5384,  0.1478, -0.9697],
         [-0.0143,  0.8320, -0.0754],
         [-0.7123,  0.8109,  0.3655]]])

In [38]:
# Luong Attention Layer
class Attn(torch.nn.Module):
    def __init__(self,method,hidden_size):
        super(Attn,self).__init__()
        self.method = method
        self.hidden_size = hidden_size
        
    def dot_score(self,hidden,encoder_output):
        # Element-wise multiply the current target decoder state with the encoder output and sum them
        return torch.sum(hidden * encoder_output , dim=2)
    
    def forward(self,hidden,encoder_outputs):
        # hidden of shape: (1,batch_size,hidden_size)
        # encoder_outputs of shape: (max_length,batch_size,hidden_size)
        # (1,batch_size,hidden_size) * (max_length,batch_size,hidden_size) = (max_length,batch_size,hidden_size)
        
        # calculate the attention weights (energies)
        attn_energies = self.dot_score(hidden,encoder_outputs) # (max_length,batch_size)
        # Transpose max_length and batch_size dimensions
        attn_energies = attn_energies.t() # (batch_size,max_length)
        # Return the softmax normalized probability scores 
        return F.softmax(attn_energies,dim=1).unsqueeze(1) # (batch_size,1,max_length)

# Designing the Decoder

In [39]:
class LuongAttnDecoderRNN(nn.Module):
    def __init__(self,attn_model,embedding,hidden_size,output_size,n_layers=1,dropout=0.1):
        super(LuongAttnDecoderRNN,self).__init__()
        self.attn_model = attn_model
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.dropout = dropout
        
        # Define Layers
        self.embedding = embedding
        self.embedding_dropout = nn.Dropout(dropout)
        self.gru = nn.GRU(hidden_size,n_layers,dropout=(0 if n_layers == 1 else dropout))
        self.concat = nn.Linear(hidden_size * 2,hidden_size)
        self.out = nn.Linear(hidden_size,output_size)
        
        self.attn_model = Attn(attn_model,hidden_size)
        
    def forward(self,input_step,last_hidden,encoder_outputs):
        # input_step: one time step (one word) of input sequence batch: shape=(1,batch_size)
        # last_hidden: final hidden layer of GRU; shape=(n_layers x num_directions,batch_size,hidden_size)
        # encoder_outputs: encoder model's output; shape=(max_length,batch_size,hidden_size)
        # Note: we run this one step (word) at a time
        
        # Get embedding of current input word
        embedded = self.embedding(input_step)
        embedded = self.embedding_dropout(embedded)
        # Forward through unidirectional GRU
        rnn_output,hidden = self.gru(embedded,last_hidden)
        # calculate attention weights from the current GRU output
        attn_weights = self.attn(rnn_output,encoder_outputs)
        # Multiply attention weights to encoder outputs to get new "weighted sum" context vector
        # (batch_size,1,max_length) bmm with (batch_size,max_length,hidden) = (batch_size,1,hidden)
        context = attn_weights.bmm(encoder_outputs.trasnpose(0,1))
        # Cocatenate weighted context vector and GRU output
        rnn_output = rnn_output.squeeze(0)
        context = context.squeeze(1)
        concat_input = torch.cat((rnn_output,context),1)
        concat_output = torch.tanh(self.concat(concat_input))
        # Predict next word using Luong eq 6.
        output = self.out(concat_output)
        output = self.F.softmax(output,dim=1)
        # Return output and final hidden state
        return output,hidden
        # ouput: softmax normalized tensor giving probabilities of each word being 
        # the correct next word in the decoded sequence
        # shape = (batch_size,voc.num_words)
        # hidden: final hidden state of GRU; shape=(n_layers X num_directions, batch_size,hidden_size) 

# Creating the Loss function

### We are done building with architecture, now let's move on the training code  
since we are dealing with batchec of padded sequences, we can not simply consider all elements of the tensor when calculating loss. We define maskNLLLoss to calculate our loss based on our decoder's output tensor, and a binary ,mask tensor describing the padding of the target tensor. This loss function calculates the average negative log likelihood of the elements that correspond to a 1 in the mask tensor. 

In [40]:
def maskNLLLoss(decoder_out,target,mask):
    nTotal = mask.sum # how many elements should we consider 
    traget = target.view(-1,1)
    # decoder_out shape: (batch_size,vocab_size) ,target_size = (batch_size,1)
    gathered_tensor = torch.gather(decoder_out,1,target)
    # Calculate the negative log likelihood Loss
    crossEntropy = -torch.log(gathered_tensor)
    # Select the non-zero elements  
    loss = crossEntropy.mask_select(mask)
    # Calculate the mean of the loss
    loss = loss.mean()
    loss = loss.to(device)
    return loss,nTotal.item()

* Before we move on to training, let's see live training and what's happening with the data

In [1]:
# Visualizing what's happening in one iteration. Only run this for visualizing.
small_batch_size = 5
batches = batch2TrainData(voc,[random.choice(pairs) for _ in range(small_batch_size)])
input_variable,length,target_variable,mask,max_target_len = batches

print("Input variable shape ",input_variable.shape)
print("lengths shape ",lengths.shape)
print("target_variable shape ",target_variable.shape)
print("mask shape ",mask.shape)
print("max_target_length ",max_target_len)

# Define the parameters 
hidden_size = 500
encoder_n_layers = 2
dropout = 0.1
attn_model = 'dot'
embedding = nn.Embedding(voc.num_words,hidden_size)

# Define the encoder and decoder
encoder = EncoderRNN(hidden_size,embedding,encoder_n_layers,dropout)
decoder = LuongAttndecoderRNN(attn_model,embedding,hidden_size,voc.num_words,decoder_n_layers,dropout)
encoder = encoder.to(device)
decoder = decoder.to(device)
# Ensure dropout layers are in train mode
encoder.train()
decder.train()


# Initialize optmizers
encoder_optimizer = optim.Adam(encoder.parameters(),lr=0.0001)
decoder_optimizer = optim.Adam(decoder.parameters(),lr=0.0001)
encoder_optimizer.zero_grad()
decoder_optmizer.zero_grad()


input_variable = input_variable.to(device)
lengths = lengths.to(device)
target_variable = target_variable.to(device)
mask = mask.to(device)

loss = 0
print_losses = []
n_totals = 0


NameError: name 'batch2TrainData' is not defined